In [56]:
import pandas as pd
import numpy as np
import cairo
import math
import ast

In [57]:
#render parameters
log_reduction = 3

color_map = {
    "scan": "grey",
    "tran1" : 'lightblue',
    "tran2" : "blue",
    "tran3" : 'darkblue',
    "init" : "green",
    "drop" : "purple",
    'rein': 'orange',
    'shift': 'red'
}

COLOURS = {
    "grey":   (0.6, 0.6, 0.6),
    "green":  (0.2, 0.7, 0.3),
    "blue":   (0.2, 0.4, 0.8),
    "purple": (0.6, 0.3, 0.7),
    'orange': (1, 0.6, 0 ),
    'darkblue': (0.05,0.05,0.6),
    'lightblue': (0.4,0.5,0.9),
    'red': (0.9,0.1,0.1)
}

In [58]:
edgelist = pd.read_csv('graph.csv', sep = '\t')

print(edgelist)

      source    target  weight  state   flux
0   (-1, -1)    (1, 0)     1.0   load  0.000
1    (20, 0)   (20, 2)     0.5   init  0.500
2    (32, 0)   (32, 2)     0.5   init  0.250
3    (80, 2)   (80, 0)     0.5   rein  0.375
4     (1, 0)   (20, 0)     1.0   scan  1.000
5    (20, 2)   (32, 2)     1.0  tran2  0.500
6    (32, 2)   (80, 2)     1.0  tran2  0.750
7    (80, 0)  (100, 0)     1.0   scan  0.625
8    (20, 0)   (32, 0)     0.5   scan  0.500
9    (32, 0)   (60, 0)     0.5   scan  0.250
10   (60, 0)   (80, 0)     1.0   scan  0.250
11  (100, 0)  (-1, -1)     1.0   drop  0.625
12   (80, 2)  (-1, -1)     0.5   drop  0.375


In [59]:
#give vert category

out = edgelist.copy()
# Extract y-coordinates
out['source'] = out['source'].apply(ast.literal_eval)
out['target'] = out['target'].apply(ast.literal_eval)

out['source_x'] = out['source'].apply(lambda v: int(v[0]))
out['target_x'] = out['target'].apply(lambda v: int(v[0]))

out['source_y'] = out['source'].apply(lambda v: int(v[1]))
out['target_y'] = out['target'].apply(lambda v: int(v[1]))

x_pos = list(out['source_x'])
x_pos.append(int(out.loc[len(out)-1, 'target_x']))
x_pos = list(dict.fromkeys(x_pos))
x_pos = sorted(x_pos)
print(x_pos)

gaps = []
for idx, x in enumerate(x_pos):
    if idx == len(x_pos) -1:
        break
    gaps.append(x_pos[idx + 1] - x_pos[idx]) 

log_gaps = [math.log(x, log_reduction) for x in gaps]
print(gaps)
log_x_pos = [x_pos[0]]
for idx, gap in enumerate(log_gaps):
    log_x_pos.append(log_x_pos[idx]+gap)

log_map = {}
for x, log_x in zip(x_pos, log_x_pos):
    log_map[x] = log_x
print(log_x_pos)
print(log_map)
# for i in gaps:
out['target_x'] = out['target_x'].map(log_map)
out['source_x'] = out['source_x'].map(log_map)



# Compute vertical direction (+1 or -1)
out.loc[out["target_y"] < 0, "target_y"] = out.loc[out["target_y"] < 0, "source_y"] + 0.5

out['vert'] = (out['target_y'] - out['source_y'] ) / (out['target_y'] - out['source_y']).abs()
out['vert'] = out['vert'].fillna(0).astype(int)

out['source_y'] = out['source_y'] * 2
out['target_y'] = out['target_y'] * 2

print(out)
edgelist = out

[-1, 1, 20, 32, 60, 80, 100]
[2, 19, 12, 28, 20, 20]
[-1, -0.3690702464285426, 2.3110736128178324, 4.572933119960747, 7.606036376265084, 10.332869404125926, 13.059702431986768]
{-1: -1, 1: -0.3690702464285426, 20: 2.3110736128178324, 32: 4.572933119960747, 60: 7.606036376265084, 80: 10.332869404125926, 100: 13.059702431986768}
      source    target  weight  state   flux   source_x   target_x  source_y  \
0   (-1, -1)    (1, 0)     1.0   load  0.000  -1.000000  -0.369070        -2   
1    (20, 0)   (20, 2)     0.5   init  0.500   2.311074   2.311074         0   
2    (32, 0)   (32, 2)     0.5   init  0.250   4.572933   4.572933         0   
3    (80, 2)   (80, 0)     0.5   rein  0.375  10.332869  10.332869         4   
4     (1, 0)   (20, 0)     1.0   scan  1.000  -0.369070   2.311074         0   
5    (20, 2)   (32, 2)     1.0  tran2  0.500   2.311074   4.572933         4   
6    (32, 2)   (80, 2)     1.0  tran2  0.750   4.572933  10.332869         4   
7    (80, 0)  (100, 0)     1.0 

/tmp/ipykernel_19001/953739991.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5 2.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  out.loc[out["target_y"] < 0, "target_y"] = out.loc[out["target_y"] < 0, "source_y"] + 0.5


In [60]:
def get_parent_edges(edge, edgelist):
    source = edge['source']
    parents = edgelist[edgelist['target'] == source]
    return parents

In [61]:
def get_sister_edge(edge, edgelist):
    source = edge['source']
    target = edge['target']
    sister = edgelist[(edgelist['source'] == source) & (edgelist['target'] != target)]
    return sister

In [62]:
out = edgelist.copy()
out['bot'] = np.nan
out['top'] = np.nan
horizontal = out[out['vert'] == 0]
print(horizontal)
for i in horizontal['state'].unique():
    for idx, edge in horizontal.iterrows():
        parent_frame = get_parent_edges(edge=edge, edgelist=out)
        num_parents = len(parent_frame)
        if num_parents == 0:
            out.loc[idx, 'bot'] = edge['source_y']
        elif num_parents == 1:
            parent_vert = parent_frame['vert'].values[0]
            print(parent_vert)
            if parent_vert != 0:
                out.loc[idx, 'bot'] = edge['source_y']
            else:
                sister_vert = get_sister_edge(edge, edgelist)['vert'].values[0]
                if sister_vert == 1:
                    out.loc[idx,'bot'] = parent_frame['bot'].values[0]
                elif sister_vert == -1:
                    print(parent_frame['bot'].values[0])
                    print(parent_frame['flux'].values[0])
                    print(edge['flux'])
                    out.loc[idx,'bot'] = parent_frame['bot'].values[0] + parent_frame['flux'].values[0] - edge['flux']
        elif num_parents == 2:
            vert = parent_frame[parent_frame['vert'] != 0]['vert'].values[0]
            print(vert)

            print(edge)
            if vert == 1:
                out.loc[idx,'bot'] = parent_frame[parent_frame['vert'] == 0]['bot'].values[0] + \
                parent_frame[parent_frame['vert'] == 0]['flux'].values[0] - \
                edge['flux']
                                
            else:
                out.loc[idx,'bot'] = parent_frame[parent_frame['vert'] == 0]['bot'].values[0]
        out.loc[idx, 'top'] = out.loc[idx, 'bot'] + out.loc[idx, 'flux']


print(out)

edgelist = out

     source    target  weight  state   flux   source_x   target_x  source_y  \
4    (1, 0)   (20, 0)     1.0   scan  1.000  -0.369070   2.311074         0   
5   (20, 2)   (32, 2)     1.0  tran2  0.500   2.311074   4.572933         4   
6   (32, 2)   (80, 2)     1.0  tran2  0.750   4.572933  10.332869         4   
7   (80, 0)  (100, 0)     1.0   scan  0.625  10.332869  13.059702         0   
8   (20, 0)   (32, 0)     0.5   scan  0.500   2.311074   4.572933         0   
9   (32, 0)   (60, 0)     0.5   scan  0.250   4.572933   7.606036         0   
10  (60, 0)   (80, 0)     1.0   scan  0.250   7.606036  10.332869         0   

    target_y  vert  bot  top  
4        0.0     0  NaN  NaN  
5        4.0     0  NaN  NaN  
6        4.0     0  NaN  NaN  
7        0.0     0  NaN  NaN  
8        0.0     0  NaN  NaN  
9        0.0     0  NaN  NaN  
10       0.0     0  NaN  NaN  
1
1
1
source        (32, 2)
target        (80, 2)
weight            1.0
state           tran2
flux             0.75
sou

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def get_child_edges(edge, edgelist):
    target = edge['target']
    children = edgelist[edgelist['source'] == target]
    return children

In [ ]:
out = edgelist.copy()
out[['left', 'right']] = [np.nan, np.nan]
vertical = out[out['vert'] != 0]
for idx, edge in vertical.iterrows():
    if edge['state'] == 'drop':
        out.loc[idx, 'bot'] = get_parent_edges(edge, edgelist)['bot'].values[0] + get_parent_edges(edge, edgelist)['flux'].values[0]
        out.loc[idx, 'top'] = out.loc[idx, 'bot'] + 0.5
        out.loc[idx, 'left'] = edge['source_x'] - (0.5 * edge['flux'])
        out.loc[idx, 'right'] = edge['source_x'] + (0.5 * edge['flux'])
        continue
    if edge['vert'] == 1:
        print(edge)
        print(get_child_edges(edge, edgelist)['bot'].values[0])
        out.loc[idx, 'top'] = get_child_edges(edge, edgelist)['bot'].values[0]
        
        out.loc[idx, 'bot'] = get_parent_edges(edge, edgelist)['bot'].values[0] + get_parent_edges(edge, edgelist)['flux'].values[0]
    elif edge['vert'] == -1:
        out.loc[idx, 'top'] = get_parent_edges(edge, edgelist)['bot'].values[0]
        out.loc[idx, 'bot'] = get_child_edges(edge, edgelist)['bot'].values[0] + get_child_edges(edge, edgelist)['flux'].values[0]

    out.loc[idx, 'left'] = edge['source_x'] - (0.5 * edge['flux'])
    out.loc[idx, 'right'] = edge['source_x'] + (0.5 * edge['flux'])



print(out)
edgelist = out


source       (1, -1)
target       (20, 0)
weight           1.0
state           scan
flux             1.0
source_x    -0.36907
target_x    2.311074
source_y          -2
target_y         0.0
vert               1
bot              NaN
top              NaN
left             NaN
right            NaN
Name: 0, dtype: object
nan


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
out = edgelist.copy()
horizontal = out[out['vert'] == 0]
for idx, edge in horizontal.iterrows():
    if len(get_parent_edges(edge, edgelist)) == 0:
        out.loc[idx, 'left'] = edge['source_x']
    elif len(get_parent_edges(edge, edgelist)) == 1:
        out.loc[idx, 'left'] = get_parent_edges(edge, out)['right'].values[0]
    else:
        out.loc[idx, 'left'] = get_parent_edges(edge, out)[get_parent_edges(edge, out)['vert'] != 0]['right'].values[0]

    if 1 in get_child_edges(edge, edgelist)['vert'].values or -1 in get_child_edges(edge, edgelist)['vert'].values:
        out.loc[idx, 'right'] = get_child_edges(edge, out)[get_child_edges(edge, out)['vert'] != 0]['left'].values[0]
    else:
        childs_vert_parent = get_parent_edges(get_child_edges(edge, edgelist).iloc[0], out)[get_parent_edges(get_child_edges(edge, edgelist).iloc[0], edgelist)['vert'] != 0]
        if len(childs_vert_parent) != 0:
            out.loc[idx, 'right'] = get_parent_edges(get_child_edges(edge, edgelist).iloc[0], out)[get_parent_edges(get_child_edges(edge, edgelist).iloc[0], edgelist)['vert'] != 0]['right'].values[0]





out['colour'] = out['state'].map(color_map)
print(out)

     source    target  weight  state  flux  source_x  target_x  source_y  \
0    (1, 0)   (20, 0)     1.0   scan  1.00 -0.369070  2.311074         0   
1   (20, 0)   (20, 2)     0.5   init  0.50  2.311074  2.311074         0   
2   (80, 2)   (80, 0)     0.5   rein  0.25  6.037907  6.037907         4   
3   (20, 2)   (80, 2)     1.0  tran2  0.50  2.311074  6.037907         4   
4   (80, 0)  (100, 0)     1.0   scan  0.75  6.037907  8.764740         0   
5   (20, 0)   (80, 0)     0.5   scan  0.50  2.311074  6.037907         0   
6  (100, 0)  (-1, -1)     1.0   drop  0.75  8.764740 -1.000000         0   
7   (80, 2)  (-1, -1)     0.5   drop  0.25  6.037907 -1.000000         4   

   target_y  vert   bot   top      left     right  colour  
0       0.0     0  0.00  1.00 -0.369070  2.061074    grey  
1       4.0     1  1.00  4.00  2.061074  2.561074   green  
2       0.0    -1  0.75  4.00  5.912907  6.162907  orange  
3       4.0     0  4.00  4.50  2.561074  5.912907    blue  
4       0.0    

In [ ]:
rectangles = out.copy()
circles = pd.DataFrame(columns=['centre_x', 'centre_y', 'radius', 'quarter', 'colour'])
for idx, rectangle in rectangles.iterrows():

    circles.loc[idx, 'centre_x'] = rectangle['left']
    circles.loc[len(rectangles)+idx, 'centre_x'] = rectangle['right']
    if rectangle['vert'] == 1:
        circles.loc[idx, 'centre_y'] = rectangle['bot']
        circles.loc[idx, 'quarter'] = 1
        if len(get_child_edges(rectangle, rectangles)) != 0:    
            circles.loc[len(rectangles)+idx, 'centre_y'] = rectangle['top']
            circles.loc[len(rectangles)+idx, 'quarter'] = 3
    elif rectangle['vert'] == 0:
        continue
    else:
        circles.loc[idx, 'centre_y'] = rectangle['top']
        circles.loc[idx, 'quarter'] = 4
        if len(get_child_edges(rectangle, rectangles)) != 0:
            circles.loc[len(rectangles)+idx, 'centre_y'] = rectangle['bot']
            circles.loc[len(rectangles)+idx, 'quarter'] = 2

    circles.loc[idx, 'radius'] = rectangle['flux']
    circles.loc[idx, 'colour'] = rectangle['colour']

    if len(get_child_edges(rectangle, rectangles)) == 0:
        continue
    child_colour = get_child_edges(rectangle, rectangles)['colour'].values[0]

    circles.loc[len(rectangles)+idx, 'radius'] = rectangle['flux']
    circles.loc[len(rectangles)+idx, 'colour'] = child_colour
circles = circles.dropna()
print(rectangles)
print(circles)

     source    target  weight  state  flux  source_x  target_x  source_y  \
0    (1, 0)   (20, 0)     1.0   scan  1.00 -0.369070  2.311074         0   
1   (20, 0)   (20, 2)     0.5   init  0.50  2.311074  2.311074         0   
2   (80, 2)   (80, 0)     0.5   rein  0.25  6.037907  6.037907         4   
3   (20, 2)   (80, 2)     1.0  tran2  0.50  2.311074  6.037907         4   
4   (80, 0)  (100, 0)     1.0   scan  0.75  6.037907  8.764740         0   
5   (20, 0)   (80, 0)     0.5   scan  0.50  2.311074  6.037907         0   
6  (100, 0)  (-1, -1)     1.0   drop  0.75  8.764740 -1.000000         0   
7   (80, 2)  (-1, -1)     0.5   drop  0.25  6.037907 -1.000000         4   

   target_y  vert   bot   top      left     right  colour  
0       0.0     0  0.00  1.00 -0.369070  2.061074    grey  
1       4.0     1  1.00  4.00  2.061074  2.561074   green  
2       0.0    -1  0.75  4.00  5.912907  6.162907  orange  
3       4.0     0  4.00  4.50  2.561074  5.912907    blue  
4       0.0    

In [ ]:
import cairo
import math


# Data bounds
x_min = min(rectangles.left.min(), circles.centre_x.min())
x_max = max(rectangles.right.max(), circles.centre_x.max())
y_min = min(rectangles.bot.min(), circles.centre_y.min())
y_max = max(rectangles.top.max(), circles.centre_y.max())

# Optional: define pixels per unit
px_per_unit = 100  # 100 pixels per 1 unit of data

# Canvas size based on data
WIDTH  = int((x_max - x_min) * px_per_unit)
HEIGHT = int((y_max - y_min) * px_per_unit)

# Scale for x and y
x_scale = WIDTH / (x_max - x_min)
y_scale = HEIGHT / (y_max - y_min)

# Use uniform scale
scale = min(x_scale, y_scale)

# Compute margins to center
x_margin = (WIDTH - scale * (x_max - x_min)) / 2
y_margin = (HEIGHT - scale * (y_max - y_min)) / 2

# Coordinate transform
def sx(x):
    return x_margin + (x - x_min) * scale

def sy(y):
    return HEIGHT - (y_margin + (y - y_min) * scale)  # invert y-axis



In [ ]:
surface = cairo.SVGSurface('graph.svg', WIDTH, HEIGHT)
ctx = cairo.Context(surface)

ctx.set_source_rgb(1, 1, 1)
ctx.paint()


In [ ]:
for _, r in rectangles.iterrows():
    x = sx(r.left)
    y = sy(r.top)
    w = sx(r.right) - sx(r.left)
    h = sy(r.bot) - sy(r.top)

    ctx.rectangle(x, y, w, h)
    ctx.set_source_rgba(*COLOURS[r.colour], 0.7)
    ctx.fill()
    # ctx.set_source_rgb(0, 0, 0)
    # ctx.stroke()


In [ ]:
QUARTER_ANGLES = {
    1: (0, math.pi/2),
    2: (math.pi/2, math.pi),
    3: (math.pi, 3*math.pi/2),
    4: (3*math.pi/2, 2*math.pi),
}

for _, a in circles.iterrows():
    cx = sx(a.centre_x)
    cy = sy(a.centre_y)
    r  = sx(a.centre_x + a.radius) - sx(a.centre_x)

    start, end = QUARTER_ANGLES[a.quarter]

    ctx.move_to(cx, cy)
    ctx.arc(cx, cy, r, start, end)
    ctx.close_path()

    ctx.set_source_rgba(*COLOURS[a.colour], 0.7)
    ctx.fill()
    # ctx.set_source_rgb(0, 0, 0)
    # ctx.stroke()


In [ ]:
surface.finish()